<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb04_Foodmart_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Per favore, non modificate il codice di questa cella
#urldb='https://drive.google.com/uc?export=download&id=1AdEOuHUAROodt60z4JMs2qptTLHiPHqL' gdrive issue with splash screen
urldb='https://dl.dropboxusercontent.com/s/ax0358bwt4poscl/db04foodmart.db.pez?dl=0'
fn='db04foodmart.db.pez'
pwd='studymat3r1al!'

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    print(zipFileName)
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=100): # eq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
Unzipping the password protected file content
db04foodmart.db.pez
File db04foodmart.db extracted
File saved in /content/db04foodmart.db


# Database Foodmart

Foodmart è una catena di supermercati (fittizia), che opera negli Stati Uniti, Canada e Messico, è composta da punti vendita la cui tipologia spazia dai piccoli negozi a supermercati di ampie dimensioni. Il database contiene i dati delle vendite, delle spese dei clienti, dei costi e delle promozioni ... e tante altre informazioni, riferiti ad un arco temporale di alcuni anni.

Nel corso delle esercitazioni, analizzeremo prevalentemente la tabella **sales_fact_1998** (e le tabelle collegate) la quale contiene i dati delle vendite effettuate dai negozi della catena nell’anno 1998.

Ogni record della tabella sales_fact_1998 riporta i dati di un prodotto acquistato da una persona.

La tabella **sales_fact_1998** è composta dai seguenti attributi:

* product_id: foreign key che identifica il prodotto venduto
* time_id: foreign key che identifica il giorno in cui è avvenuta la vendita
* customer_id: foreign key che identifica il cliente
* promotion_id: foreign key che identifica l’eventuale promozione applicata
* store_id: foreign key che identifica il negozio in cui è avvenuta la vendita
* store_sales: prezzo unitario di vendita applicato
* store_cost: stima del costo unitario che il supermercato ha sostenuto per acquistare il prodotto, trasportarlo, …
* unit_sales: quantità di prodotto venduta

Del database foodmart fanno parte anche le tabelle elencate qui di seguito (tra parentesi le traduzioni in italiano):
* product (prodotti)
* product_class (classi di prodotti)
* store (negozi)
* time_by_day (informazioni sui giorni dell’anno)
* promotion (promozioni)
* customer (clienti)
* region (regione)

queste ultime tabelle contengono informazioni extra sulle vendite effettuate, per esempio le tipologie di prodotti venduti, il negozio che ha effettuato la vendita, i dati del cliente che ha effettuato l’acquisto, le date e così via. Le informazioni delle varie tabelle sono collegate alla tabella sales_fact_1998 e tra di loro per mezzo di chiavi. es.: product_id, time_id, ...



# Esercizi

Svolgete quanto richiesto nei punti seguenti.

# (EDB04) 1

Negli esercizi successivi utilizzeremo le seguenti tabelle: **sales_fact_1998**, **product**, **produc_class**, **store**, **time_by_day**, **promotion**, **customer**, **region**

Scrivete su un foglio di carta il nome di queste tabelle, per ogni tabella individuate (se presenti):

* la chiave primaria (o le chiavi primarie se più attributi sono coinvolti)

* i collegamenti con le altre tabelle (individuate cioè le foreign key). Suggerimento: tracciate delle frecce per collegare le foreign key con le corrispondenti chiavi primarie

Questo foglio sarà utilizzato negli esercizi successivi.

NB: L'obiettivo di questo esercizio è duplice: da un lato dovete imparare a costruire (velocemente) uno schema logico del database in assenza di documentazione. Dall'altro, lo scrivere le tabelle e le chiavi su carta, servirà a voi solo come riferimento per svolgere gli esercizi indicati di seguito. Vi suggeriamo di impiegare un tempo adeguato per questo task, ma senza andare oltre i 10 minuti.

In [2]:
rq("""
select * from sales_fact_1998 limit 10;
""")

,product_id,time_id,customer_id,promotion_id,store_id,store_sales,store_cost,unit_sales
0,173,748,2094,54,1,4.29,1.8447,3
1,1119,748,2094,54,1,9.51,3.5187,3
2,1242,748,2094,54,1,7.92,2.8512,4
3,460,748,2094,54,1,6.44,2.7048,4
4,104,748,2094,54,1,11.67,3.9678,3
5,27,748,2094,54,1,7.95,3.8160,3
6,67,748,1277,54,1,7.44,2.9016,4
7,217,748,1277,54,1,2.72,0.8432,4
8,666,748,1745,54,1,2.67,1.3350,3
9,1122,748,1745,54,1,2.22,0.7770,3


10 records returned


# (EDB04) 2

Che informazioni memorizza la tabella time_by_day? Qual è la sua utilità? Come è collegata a sales_fact_1998?

Scrivete la vostra risposta in questa cella (doppio click per entrare in modalità editing)

In [3]:
# Potete usare questa cella per interrogare la tabella time_by_day.
# Si consiglia di limitare la visualizzazione ai primi 10 record
rq("""
SELECT *
FROM time_by_day
limit 10;
""")

,time_id,the_date,the_day,the_month,the_year,day_of_month,week_of_year,month_of_year,quarter,fiscal_period
0,738,1998-01-07 00:00:00,Wednesday,January,1998,7,4,1,Q1,None
1,739,1998-01-08 00:00:00,Thursday,January,1998,8,4,1,Q1,None
2,740,1998-01-09 00:00:00,Friday,January,1998,9,4,1,Q1,None
3,741,1998-01-10 00:00:00,Saturday,January,1998,10,4,1,Q1,None
4,742,1998-01-11 00:00:00,Sunday,January,1998,11,5,1,Q1,None
5,743,1998-01-12 00:00:00,Monday,January,1998,12,5,1,Q1,None
6,744,1998-01-13 00:00:00,Tuesday,January,1998,13,5,1,Q1,None
7,745,1998-01-14 00:00:00,Wednesday,January,1998,14,5,1,Q1,None
8,746,1998-01-15 00:00:00,Thursday,January,1998,15,5,1,Q1,None
9,747,1998-01-16 00:00:00,Friday,January,1998,16,5,1,Q1,None


10 records returned


# (EDB04) 3

Visualizzate product_id, nome prodotto e la data delle vendite contenute nella tabella sales_fact_1998.

La data deve essere visualizzata in un formato nel quale si riesca a distinguere giorno, mese e anno;

Il nome del prodotto e la data vanno recuperate da tabelle diversa da sales_fact_1998.

I risultati devono essere ordinati per data e per nome prodotto.

Limitate la visualizzazione ai primi 20 record (limit 20 in fondo alla query)

Nota: la prima riga del risultato dovrebbe essere: 1006; American Foot-Long Hot Dogs; 1998-01-01 00:00:00.


In [6]:
rq("""

SELECT sf.product_id,
      p.product_name,
      t.the_date

FROM sales_fact_1998 AS sf,
      product AS p,
      time_by_day AS t

WHERE (sf.product_id = p.product_id)
      AND (sf.time_id = t.time_id)

ORDER BY the_date, product_name
limit 20;
""")

,product_id,product_name,the_date
0,1006,American Foot-Long Hot Dogs,1998-01-01 00:00:00
1,1464,Amigo Lox,1998-01-01 00:00:00
2,571,Atomic White Chocolate Bar,1998-01-01 00:00:00
3,932,BBB Best Apple Jam,1998-01-01 00:00:00
4,921,BBB Best Canola Oil,1998-01-01 00:00:00
5,936,BBB Best Extra Chunky Peanut Butter,1998-01-01 00:00:00
6,910,BBB Best Sesame Oil,1998-01-01 00:00:00
7,228,Best Choice Cheese Crackers,1998-01-01 00:00:00
8,248,Best Choice Dried Dates,1998-01-01 00:00:00
9,236,Best Choice Fondue Mix,1998-01-01 00:00:00


20 records returned


# (EDB04) 4

Per ogni record della tabella sales_fact_1998 visualizzate il time_id, il product_id, le quantità vendute, il ricavo unitario, il costo unitario e il guadagno conseguito.

Vi ricordiamo che: **guadagno=(ricavo_unitario - costo_unitario) * quantità**. Si ricorda che nella tabella sales_fact_1998 gli attributi **store_sales**, **store_cost** e **unit_sales** rappresentano rispettivamente: **ricavo_unitario**, **costo_unitario** e **quantità_vendute**.

Ordinate il risultato in base al time_id e al product_id. Vi suggeriamo di limitare l'output ai primi 10 record


Nota: la prima riga di risultato dovrebbe essere: product_id 4, unit_sales 3, guadagno 18.0180

In [10]:
rq("""

SELECT time_id,
      product_id,
      unit_sales,
      store_sales,
      store_cost,
      (store_sales-store_cost)*unit_sales

FROM sales_fact_1998

ORDER BY time_id, product_id
limit 10;

""")

,time_id,product_id,unit_sales,store_sales,store_cost,(store_sales-store_cost)*unit_sales
0,732,4,3,10.92,4.9140,18.0180
1,732,11,3,10.53,4.9491,16.7427
2,732,12,3,4.62,2.0790,7.6230
3,732,14,2,3.86,1.4282,4.8636
4,732,16,3,11.49,5.1705,18.9585
5,732,23,3,5.25,1.6800,10.7100
6,732,46,3,10.32,4.1280,18.5760
7,732,50,4,15.80,7.7420,32.2320
8,732,56,4,2.84,1.1360,6.8160
9,732,59,3,4.20,1.6380,7.6860


10 records returned


# (EDB04) 5

Visualizzate l’elenco dei prodotti venduti nel corso dell’anno. Nella tabella risultato deve apparire l’id e il nome per esteso del prodotto.

Ogni prodotto anche se venduto più volte deve apparire una sola volta nel risultato. (dovete usare la clausola GROUP BY).

I prodotti devono essere mostrati in ordine crescente rispetto al nome.

Nota, la prima riga di risultato dovrebbe essere: 351; ADJ Rosy Sunglasses


In [3]:
rq("""

SELECT sf.product_id,
      p.product_name

FROM sales_fact_1998 AS sf,
      product AS p

WHERE sf.product_id = p.product_id

GROUP BY sf.product_id
ORDER BY p.product_name;

""")

,product_id,product_name
0,351,ADJ Rosy Sunglasses
1,664,Akron City Map
2,663,Akron Eyeglass Screwdriver
3,1003,American Beef Bologna
4,1001,American Chicken Hot Dogs
5,1008,American Cole Slaw
6,1010,American Corned Beef
7,1006,American Foot-Long Hot Dogs
8,1004,American Low Fat Bologna
9,1007,American Low Fat Cole Slaw


Too many records in the result, returned only the first 100


# (EDB04) 6

Per ogni prodotto, calcolate il totale dei costi di acquisto di tutte le vendite (ogni prodotto anche se venduto più volte, deve apparire una sola volta nella tabella risultato);

costo di una singola vendita = costo_unitario * qt_venduta.

Nel risultato va visualizzato il nome del prodotto.

Nota, nel risultato ci sarà: product_id 1, Washington Berry Juice; totale costi acquisto 519.7545

In [7]:
rq("""

SELECT p.product_name,
      sf.product_id,
      SUM(sf.store_cost * sf.unit_sales) AS COSTO_TOT

FROM sales_fact_1998 AS sf,
      product AS p

WHERE sf.product_id = p.product_id

GROUP BY sf.product_id;

""")

,product_name,product_id,COSTO_TOT
0,Washington Berry Juice,1,519.7545
1,Washington Mango Drink,2,260.0952
2,Washington Strawberry Drink,3,277.3362
3,Washington Cream Soda,4,1366.4560
4,Washington Diet Soda,5,824.2941
5,Washington Cola,6,486.6570
6,Washington Diet Cola,7,881.3448
7,Washington Orange Juice,8,906.6036
8,Washington Cranberry Juice,9,945.2036
9,Washington Apple Juice,10,477.2620


Too many records in the result, returned only the first 100


# (EDB04) 7

Per ogni prodotto calcolate il totale dei guadagni (ogni prodotto anche se venduto più volte, deve apparire una sola volta).

Nel risultato va visualizzato il nome del prodotto. Il guadagno associato ad una singola vendita può essere calcolato con la formula (ricavo_unitario – costo_unitario)*qt_venduta.

Nota, il prodotto con product_id 12 ha questi dati: Jeffers Oatmeal; 1017.78600000


In [10]:
rq("""

SELECT p.product_name,
      sf.product_id,
      SUM( (sf.store_sales - sf.store_cost) * sf.unit_sales) AS RICAVO_TOT

FROM sales_fact_1998 AS sf

JOIN product AS p
    ON sf.product_id = p.product_id

GROUP BY sf.product_id;

""")

,product_name,product_id,RICAVO_TOT
0,Washington Berry Juice,1,785.5455
1,Washington Mango Drink,2,409.6048
2,Washington Strawberry Drink,3,424.8438
3,Washington Cream Soda,4,1953.2240
4,Washington Diet Soda,5,1205.8359
5,Washington Cola,6,695.5430
6,Washington Diet Cola,7,1391.9652
7,Washington Orange Juice,8,1346.6964
8,Washington Cranberry Juice,9,1377.9964
9,Washington Apple Juice,10,721.2180


Too many records in the result, returned only the first 100


# (EDB04) 8

Ordinate l’elenco del punto precedente in ordine decrescente in base al guadagno.

Nota, i dati del prodotto che ha fatto guadagnare di più sono: product_id 274, Great Pumpernickel Bread, gudadagno 3166.3214


In [11]:
rq("""

SELECT p.product_name,
      sf.product_id,
      SUM( (sf.store_sales - sf.store_cost) * sf.unit_sales) AS RICAVO_TOT

FROM sales_fact_1998 AS sf

JOIN product AS p
    ON sf.product_id = p.product_id

GROUP BY sf.product_id
ORDER BY RICAVO_TOT DESC;

""")

,product_name,product_id,RICAVO_TOT
0,Great Pumpernickel Bread,274,3166.3214
1,Super Extra Chunky Peanut Butter,309,2988.5634
2,Horatio Frosted Cookies,1180,2971.0200
3,High Top New Potatos,175,2969.1200
4,Hilltop Mint Mouthwash,1380,2961.5582
5,Even Better Large Curd Cottage Cheese,993,2946.4235
6,Fabulous Strawberry Drink,941,2924.2640
7,Better Chicken Soup,330,2908.2258
8,Horatio Low Fat Chips,1178,2886.8616
9,Modell Rye Bread,1525,2881.2009


Too many records in the result, returned only the first 100


#(EDB04) 9

Individuate i supermercati che hanno venduto più quantità di prodotti (create una elenco in ordine decrescente).

Nota, il primo della lista è: Store 12 con qt pari a 37680.0000


In [12]:
rq("""
SELECT st.store_name,
      SUM(sf.unit_sales) as QT_TOT

FROM sales_fact_1998 AS sf

JOIN store AS st
    ON sf.store_id = st.store_id

GROUP BY st.store_id
ORDER BY QT_TOT DESC;
""")

,store_name,QT_TOT
0,Store 12,37680
1,Store 8,37143
2,Store 19,36643
3,Store 17,35444
4,Store 13,35346
5,Store 21,34343
6,Store 15,26672
7,Store 16,26164
8,Store 11,25266
9,Store 10,24696


24 records returned


#(EDB04) 10

Individuate i supermercati che hanno venduto la minor quantità di prodotti. Visualizzate una lista in ordine crescente.

Nota, il primo della lista è: Store 2 con qt pari a 1984


In [13]:
rq("""
SELECT st.store_name,
      SUM(sf.unit_sales) as QT_TOT

FROM sales_fact_1998 AS sf

JOIN store AS st
    ON sf.store_id = st.store_id

GROUP BY st.store_id
ORDER BY QT_TOT;
""")

,store_name,QT_TOT
0,Store 2,1984
1,Store 14,2027
2,Store 5,2124
3,Store 22,2244
4,Store 20,9514
5,Store 23,9710
6,Store 18,10070
7,Store 9,10880
8,Store 6,22707
9,Store 1,23226


24 records returned


#(EDB04) 11

Individuate le tipologie di prodotto che hanno fatto guadagnare di più (per le tipologie fate riferimento all’attributo product_category della tabella product_class). Visualizzate i risultati in ordine decrescente.

Nota, la prima categoria è: Vegetables; 267508.5067

In [14]:
rq("""
SELECT pc.product_category,
      SUM( (sf.store_sales - sf.store_cost)*sf.unit_sales ) as RICAVO_TOT

FROM sales_fact_1998 AS sf

JOIN product AS p
    ON sf.product_id = p.product_id

JOIN product_class AS pc
    ON p.product_class_id = pc.product_class_id

GROUP BY pc.product_category
ORDER BY RICAVO_TOT DESC;
""")

,product_category,RICAVO_TOT
0,Vegetables,267508.5067
1,Snack Foods,258630.3575
2,Dairy,141175.4869
3,Meat,115581.0941
4,Fruit,108791.4612
5,Jams and Jellies,88584.9492
6,Bread,63948.7259
7,Electrical,62239.9086
8,Baking Goods,61038.5203
9,Breakfast Foods,60419.9306


45 records returned


#(EDB04) 12

Individuate i prodotti più venduti (in termini di quantità), nella regione del ‘Central West’ (per la regione, fate riferimento all’attributo sales_region della tabella region).

Nota, il più venduto è: product_id 391, Urban Large Eggs, qt 10

In [19]:
rq("""

SELECT sf.product_id,
      SUM(sf.unit_sales) AS TOT_QT

FROM sales_fact_1998 AS sf

JOIN store AS st
  ON sf.store_id = st.store_id

JOIN region AS rg
  ON st.region_id = rg.region_id

WHERE rg.sales_region = "Central West"

GROUP BY sf.product_id
ORDER BY TOT_QT DESC


""")

,product_id,TOT_QT
0,391,10
1,377,8
2,584,7
3,850,7
4,977,7
5,1052,7
6,186,6
7,474,6
8,531,6
9,547,6


Too many records in the result, returned only the first 100


#(EDB04) 13
Individuate i prodotti più venduti (in termini di quantità), negli USA (per la nazione fate riferimento all’attributo sales_country della tabella region).

Nota, il primo della lista è: product_id 876, Thresher Mint Chocolate Bar, qt 268

In [21]:
rq("""

SELECT sf.product_id,
      SUM(sf.unit_sales) AS TOT_QT

FROM sales_fact_1998 AS sf

JOIN store AS st
  ON sf.store_id = st.store_id

JOIN region AS rg
  ON st.region_id = rg.region_id

WHERE rg.sales_country = "USA"

GROUP BY sf.product_id
ORDER BY TOT_QT DESC


""")

,product_id,TOT_QT
0,876,268
1,277,255
2,319,250
3,386,247
4,426,242
5,655,239
6,717,237
7,1352,235
8,230,234
9,444,234


Too many records in the result, returned only the first 100


#(EDB04) 14

Calcolate il guadagno totale di tutti i prodotti venduti nei mesi di aprile e maggio 1998.

Nota: risultato 377409.6824


In [24]:
rq("""

SELECT SUM( (sf.store_sales - sf.store_cost) * sf.unit_sales ) AS RICAVO_TOT

FROM sales_fact_1998 AS sf

JOIN time_by_day AS tm
  ON tm.time_id = sf.time_id

WHERE (tm.month_of_year = 4 OR tm.month_of_year = 5);
""")

,RICAVO_TOT
0,377409.6824


1 records returned


#(EDB04) 15

Individuate i clienti che hanno speso di meno in tutto il 1998 (è sufficiente visualizzare i clienti in ordine crescente rispetto alle spese effettuate).

Nota, cliente che ha speso meno: customer_id 2383


In [26]:
rq("""
SELECT sf.customer_id,
      SUM(store_sales*unit_sales) AS spesa_tot

FROM sales_fact_1998 AS sf

GROUP BY customer_id
ORDER BY spesa_tot;
""")

,customer_id,spesa_tot
0,2383,1.42
1,5885,1.69
2,8874,1.93
3,10023,2.11
4,8641,2.35
5,6255,2.41
6,7803,2.41
7,9994,2.53
8,6325,2.59
9,2386,2.62


Too many records in the result, returned only the first 100


#(EDB04) 16

Individuate i clienti che hanno speso di più in tutto il 1998 (è sufficiente visualizzare i clienti in ordine decrescente rispetto alle spese effettuate).

Nota, cliente che ha speso di più: customer_id 5466


In [27]:
rq("""
SELECT sf.customer_id,
      SUM(store_sales*unit_sales) AS spesa_tot

FROM sales_fact_1998 AS sf

GROUP BY customer_id
ORDER BY spesa_tot DESC;
""")

,customer_id,spesa_tot
0,5466,6303.24
1,7836,5836.34
2,4226,5760.66
3,1770,5720.30
4,4026,5655.51
5,7633,5407.04
6,4386,5095.81
7,4953,4970.48
8,4579,4759.52
9,1612,4745.41


Too many records in the result, returned only the first 100


#(EDB04) 17

Individuate la città (attributo sales_city della tabella region) nella quale sono stati incassati più soldi nel 1998 (l’incasso è l’equivalente del ricavo).

Effettuate il group by sull’attributo region.sales_city, non su region.region_id. Inoltre, tra gli attributi della tabella region visualizzate solamente l’attributo sales_city nel risultato finale (niente region_id, sales_district, ...).

Nota: Hidalgo con 336305.4600

In [30]:
rq("""
SELECT rg.sales_city,
      SUM((sf.store_sales)*sf.unit_sales) AS RICAVO_TOT

FROM sales_fact_1998 AS sf

JOIN store AS st
  ON st.store_id = sf.store_id

JOIN region AS rg
  ON rg.region_id = st.region_id

GROUP BY rg.sales_city
ORDER BY RICAVO_TOT DESC;
""")

,sales_city,RICAVO_TOT
0,Hidalgo,336305.46
1,Merida,265264.47
2,Vancouver,261187.75
3,Tacoma,251705.35
4,Salem,249343.36
5,San Andres,240343.31
6,Seattle,188940.70
7,Spokane,185139.48
8,Portland,179921.38
9,San Diego,174050.52


23 records returned


# (EDB04) 18

Individuate le 5 sales_region in cui la catena di supermercati ha guadagnato di più.

Nota: risultato: North West, Mexico Central, South West, Canada West, Mexico South


In [32]:
rq("""
SELECT rg.sales_region,
      SUM((sf.store_sales-sf.store_cost)*sf.unit_sales) AS GUADAGNO_TOT

FROM sales_fact_1998 AS sf

JOIN store AS st
  ON st.store_id = sf.store_id

JOIN region AS rg
  ON rg.region_id = st.region_id

GROUP BY rg.sales_region
ORDER BY GUADAGNO_TOT DESC
limit 5;
""")

,sales_region,GUADAGNO_TOT
0,North West,786230.3148
1,Mexico Central,594823.0294
2,South West,300792.5516
3,Canada West,197158.2867
4,Mexico South,158989.1902


5 records returned
